In [2]:
import sys
import pandas as pd
import dateutil
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut

import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
tf.config.experimental_run_functions_eagerly(True)

sys.path.append('../')
from utilities.ANN_functions import root_mean_squared_error, r2_keras_loss, r2_keras
from utilities.ANN_functions import create_loso_model, create_loyo_model, create_lsyso_model

import warnings
warnings.filterwarnings('ignore')
date_parser = lambda x: dateutil.parser.parse(x, ignoretz=True)

from dask.distributed import Client
client = Client()
#client = Client(n_workers=2, threads_per_worker=1, memory_limit='1GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 7.67 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46883,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 7.67 GiB
Comm: tcp://127.0.0.1:36731,Total threads: 1
Dashboard: http://127.0.0.1:39173/status,Memory: 1.92 GiB
Nanny: tcp://127.0.0.1:41515,


In [7]:
cs_file = '../data/SMB_input_2011_2015.csv'

df = pd.read_csv(cs_file,
   delimiter='\t', index_col=['Date/Time'],
    parse_dates=['Date/Time'], na_values='NAN',date_parser=date_parser)

In [8]:
X = (df.drop(['Diff', 'Event'], axis=1)).to_numpy()
y = (df['Diff'].copy()).to_numpy()

In [9]:
loso = LeaveOneGroupOut()
groups = df['Event'].values

In [10]:
x, n_features = X.shape

In [11]:
path_ann = '../ANN/LOSO/CV/'

In [13]:
def run_loso():
    # Leave-One-Stake-Out
    i = 1
    test_rmse = []
    for train_index, test_index in loso.split(X, y, groups):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = create_loso_model(n_features, final=False)
        es = EarlyStopping(monitor='val_loss', mode='min', min_delta=0.01, patience=1000)
        mc = ModelCheckpoint(path_ann + 'stake_'+str(i)+'_model.h5', monitor='val_loss', mode='min',
                             save_best_only=True, verbose=1)

        history = model.fit(X_train, y_train, validation_data = (X_test, y_test), 
                            epochs=2000, batch_size = 11, verbose=1, callbacks=[es, mc])

        best_model = load_model(path_ann  + 'stake_'+str(i)+'_model.h5',
                                custom_objects={"r2_keras": r2_keras, 
                                                "r2_keras_loss": r2_keras_loss, 
                                                "root_mean_squared_error": root_mean_squared_error})

        score = best_model.evaluate(X_test, y_test)

        test_rmse.append(score[0])

        i= i+1

In [14]:
client.submit(run_loso)

<Future: pending, key: run_loso-90fe5f02e4b697091831f78fe4ec51ad>

2022-07-11 14:17:01.339046: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/christian/miniconda3/envs/DL_SMB/lib/python3.10/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/2000
78/88 [=========================>....] - ETA: 0s - loss: 0.3125 - r2_keras: -1.2757 
Epoch 1: val_loss improved from inf to 2.16288, saving model to ANN/LOSO/CV/stake_1_model.h5
88/88 [==============================] - 4s 8ms/step - loss: 0.3050 - r2_keras: -17006.9082 - val_loss: 2.1629 - val_r2_keras: -7321196.0000
Epoch 2/2000
88/88 [==============================] - ETA: 0s - loss: 0.2532 - r2_keras: -13.0262
Epoch 2: val_loss improved from 2.16288 to 0.38892, saving model to ANN/LOSO/CV/stake_1_model.h5
88/88 [==============================] - 0s 3ms/step - loss: 0.2532 - r2_keras: -13.0262 - val_loss: 0.3889 - val_r2_keras: -3943.0449
Epoch 3/2000
71/88 [=======================>......] - ETA: 0s - loss: 0.2332 - r2_keras: -0.2021
Epoch 3: val_loss did not improve from 0.38892
88/88 [==============================] - 0s 3ms/step - loss: 0.2396 - r2_keras: -0.3137 - val_loss: 0.4678 - val_r2_keras: -165096.4375
Epoch 4/2000
66/88 [=====================>........] - ETA:

In [ ]:
print(test_rmse)

In [ ]:
df['Year'] = df.index.year

In [ ]:
loyo = LeaveOneGroupOut()
groups = df['Year'].values

In [ ]:
X = (df.drop(['Diff', 'Event', 'Year'], axis=1)).to_numpy()
y = (df['Diff'].copy()).to_numpy()

In [ ]:
x, n_features = X.shape

In [ ]:
path_ann = '../ANN/LOYO/CV/'

In [ ]:
# Leave-One-Year-Out
i = 1
test_rmse = []
for train_index, test_index in loyo.split(X, y, groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = create_loyo_model(n_features)
    es = EarlyStopping(monitor='val_loss', mode='min', min_delta=0.01, patience=1000)
    mc = ModelCheckpoint(path_ann + 'year_'+str(i)+'_model.h5', monitor='val_loss', mode='min',
                         save_best_only=True, verbose=1)

    history = model.fit(X_train, y_train, validation_data = (X_test, y_test), 
                        epochs=2000, batch_size = 5, verbose=1, callbacks=[es, mc])
    
    best_model = load_model(path_ann  + 'year_'+str(i)+'_model.h5',
                            custom_objects={"r2_keras": r2_keras, 
                                            "r2_keras_loss": r2_keras_loss, 
                                            "root_mean_squared_error": root_mean_squared_error})

    score = best_model.evaluate(X_test, y_test)

    test_rmse.append(score[0])
    i= i+1

In [ ]:
print(test_rmse)

In [ ]:
X = (df.drop(['Diff', 'Event', 'Year'], axis=1)).to_numpy()
y = (df['Diff'].copy()).to_numpy()

In [ ]:
groups = df['Event'].values
years = df.drop_duplicates(subset=['Year'])['Year'].values

In [ ]:
x, n_features = X.shape

In [ ]:
path_ann = '../ANN/LOYSO/CV/'

In [ ]:
i = 1 
test_rmse = []
for year, (train_index, test_index) in zip(years, loso.split(X, y, groups)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = create_lsyso_model(n_features, final=False)
    es = EarlyStopping(monitor='val_loss', mode='min', min_delta=0.01, patience=1000)
    mc = ModelCheckpoint(path_ann + 'year_stake_'+str(i)+'_model.h5', monitor='val_loss', mode='min',
                         save_best_only=True, verbose=1)

    history = model.fit(X_train, y_train, validation_data = (X_test, y_test), 
                        epochs=2000, batch_size = 5, verbose=1, callbacks=[es, mc])
    
    best_model = load_model(path_ann  + 'year_stake_'+str(i)+'_model.h5',
                            custom_objects={"r2_keras": r2_keras, 
                                            "r2_keras_loss": r2_keras_loss, 
                                            "root_mean_squared_error": root_mean_squared_error})

    score = best_model.evaluate(X_test, y_test)

    test_rmse.append(score[0])
    i= i+1


In [ ]:
print(test_rmse)